<a href="https://colab.research.google.com/github/LeeGitaek/Kaggle_Solving/blob/master/watermelon_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip uninstall --y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 3.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=0911cb171471e5bdb784e300bc07d6a4d01811c10b11d3a66dc380c85b73aabc
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 68 Jun 28 04:56 kaggle.json


In [3]:
!kaggle competitions download -c 18011765watermelon-price

  0% 0.00/12.4k [00:00<?, ?B/s]
100% 12.4k/12.4k [00:00<00:00, 10.7MB/s]


In [4]:
!unzip 18011765watermelon-price.zip

Archive:  18011765watermelon-price.zip
  inflating: submit_sample.csv       
  inflating: test_watermelon_price.csv  
  inflating: train_water_melon_price.csv  


In [6]:
import pandas as pd
import numpy as np
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn as nn 
import torch.optim as optim

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [11]:
train = pd.read_csv('train_water_melon_price.csv')
print(train.head(10))
print(train.info())

         date  sunshineHr  avgTemp  ...  minTemp  avgRainfall  avgPrice
0  2010-01-04         9.6     -1.8  ...    -16.8          5.9     16129
1  2010-01-05         9.6     -5.2  ...    -21.5          0.7     16557
2  2010-01-06         9.6     -7.3  ...    -26.8          0.3     16129
3  2010-01-07         9.6     -6.7  ...    -25.8          0.0     16350
4  2010-01-08         9.7     -5.6  ...    -23.7          0.0     16350
5  2010-01-11         9.7     -1.5  ...    -16.3          0.0     16767
6  2010-01-12         9.7     -5.0  ...    -18.9          0.4     16767
7  2010-01-13         9.8     -8.3  ...    -23.5          0.7     17017
8  2010-01-14         9.8     -6.4  ...    -24.8          0.1     17017
9  2010-01-15         9.8     -2.6  ...    -18.7          0.1     17183

[10 rows x 9 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 749 entries, 0 to 748
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----

In [13]:
test = pd.read_csv('test_watermelon_price.csv')
print(test.head(10))
print(test.info())

         date  sunshineHr  avgTemp  ...  avgminTemp  minTemp  avgRainfall
0  2013-01-02         9.6     -6.0  ...       -11.2    -22.8          0.4
1  2013-02-01        10.3      8.6  ...         4.8     -1.9         31.3
2  2013-03-04        11.5      3.4  ...        -2.7     -8.0          0.0
3  2013-04-01        12.6      9.2  ...         1.1     -3.8          0.0
4  2013-05-02        13.8     11.8  ...         6.7      1.7          1.1
5  2013-06-03        14.7     20.1  ...        14.2      5.8          0.0
6  2013-07-01        14.8     25.4  ...        21.1     15.4          0.0
7  2013-08-01        14.1     27.2  ...        23.8     18.3          3.6

[8 rows x 8 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         8 non-null      object 
 1   sunshineHr   8 non-null      float64
 2   avgTemp      8 non-null      float64
 3

In [48]:
learning_rate = 0.01
training_epoch = 2000
batch_size = 50

In [49]:
x_train = train.iloc[:,1:-1]
y_train = train.iloc[:,[-1]]

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train = torch.FloatTensor(x_train)
y_train = torch.FloatTensor(y_train)

print(x_train.shape)
print(y_train.shape)

torch.Size([749, 7])
torch.Size([749, 1])


In [50]:
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)



In [51]:
class MishFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)
        return x * torch.tanh(F.softplus(x))   # x * tanh(ln(1 + exp(x)))

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_variables[0]
        sigmoid = torch.sigmoid(x)
        tanh_sp = torch.tanh(F.softplus(x)) 
        return grad_output * (tanh_sp + x * sigmoid * (1 - tanh_sp * tanh_sp))

class Mish(nn.Module):
    def forward(self, x):
        return MishFunction.apply(x)

def to_Mish(model):
    for child_name, child in model.named_children():
        if isinstance(child, nn.ReLU):
            setattr(model, child_name, Mish())
        else:
            to_Mish(child)

In [52]:
linear1 = nn.Linear(7,8,bias=True)
linear2 = nn.Linear(8,8,bias=True)
linear3 = nn.Linear(8,8,bias=True)
linear4 = nn.Linear(8,8,bias=True)
linear5 = nn.Linear(8,1,bias=True)
mish = Mish() # activation function


In [53]:
nn.init.kaiming_normal_(linear1.weight)
nn.init.kaiming_uniform_(linear2.weight)
nn.init.kaiming_uniform_(linear3.weight)
nn.init.kaiming_normal_(linear4.weight)
nn.init.kaiming_uniform_(linear5.weight)


Parameter containing:
tensor([[ 0.6276,  0.1401,  0.6147,  0.1417, -0.6245, -0.3665, -0.3715, -0.6858]],
       requires_grad=True)

In [54]:
model = torch.nn.Sequential(
    linear1,mish,
    linear2,mish,
    linear3,mish,
    linear4,mish,
    linear5
).to(device)

In [55]:
loss = nn.MSELoss().to(device)
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [56]:
total_batch = len(data_loader)

for epoch in range(training_epoch):
    avg_cost = 0
    for X,Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = loss(hypothesis,Y)
        cost.backward()
        optimizer.step()


        avg_cost += cost/total_batch
    
    if epoch % 10 == 0:  
        print('Epoch:', '%d' % (epoch ), 'Cost =', '{:.9f}'.format(avg_cost))
print('Learning Finished')

Epoch: 0

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  if __name__ == '__main__':


 Cost = 381654592.000000000
Epoch: 10 Cost = 16153669.000000000
Epoch: 20 Cost = 15098420.000000000
Epoch: 30 Cost = 14467705.000000000
Epoch: 40 Cost = 13947405.000000000
Epoch: 50 Cost = 13827656.000000000
Epoch: 60 Cost = 13414179.000000000
Epoch: 70 Cost = 13077185.000000000
Epoch: 80 Cost = 13046451.000000000
Epoch: 90 Cost = 12991767.000000000
Epoch: 100 Cost = 13119215.000000000
Epoch: 110 Cost = 12714568.000000000
Epoch: 120 Cost = 12739696.000000000
Epoch: 130 Cost = 12500649.000000000
Epoch: 140 Cost = 12418009.000000000
Epoch: 150 Cost = 12261992.000000000
Epoch: 160 Cost = 12162029.000000000
Epoch: 170 Cost = 12444222.000000000
Epoch: 180 Cost = 12011081.000000000
Epoch: 190 Cost = 12002901.000000000
Epoch: 200 Cost = 11819812.000000000
Epoch: 210 Cost = 11890699.000000000
Epoch: 220 Cost = 12075926.000000000
Epoch: 230 Cost = 11897220.000000000
Epoch: 240 Cost = 12239332.000000000
Epoch: 250 Cost = 11823473.000000000
Epoch: 260 Cost = 11532607.000000000
Epoch: 270 Cost = 1

In [57]:
test_data = test.iloc[:,1:]
test_data = np.array(test_data)
test_data = torch.FloatTensor(test_data).to(device)

with torch.no_grad():
    predict = model(test_data)

predict

tensor([[18788.8906],
        [19067.3984],
        [22644.0332],
        [21807.7676],
        [18758.5898],
        [16689.5898],
        [15090.7832],
        [20061.3652]], device='cuda:0')

In [58]:
correct_prediction = predict.cpu().numpy().reshape(-1,1)

In [59]:
submit = pd.read_csv('submit_sample.csv')
for i in range(len(correct_prediction)):
  submit['Expected'][i]=correct_prediction[i].item()
submit.to_csv('submit.csv', mode = 'w', index = False, header = True)
submit


,Id,Expected
0,1,18788
1,2,19067
2,3,22644
3,4,21807
4,5,18758
5,6,16689
6,7,15090
7,8,20061


In [60]:
!kaggle competitions submit -c 18011765watermelon-price -f submit.csv -m "14010974_이기택"

100% 76.0/76.0 [00:00<00:00, 95.0B/s]
Successfully submitted to SejongAI.텀프로젝트.[수박가격예측].watermelon_price